# 本セクションの目次

# クイックスタートKafkaとPyspark

1. 

# kafkaのトピックを作成します
今回は3つのトピックを作成します

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic1 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic2 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic3 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

## replication-factor/partitions　
いわゆる冗長化の設定。
レプリケーションは何台に
パーティションは何個？
という意味の冗長化。

つまりreplication-factor 2でpartitons3であれば2台のノードにまたがってパーティションを3つ（1台に２つ、もう一台に1つ）  
という設定になる。

## bootstrap-server 
データの受付口（ブローカーのIPとポートを指定する）


In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# kafkaにデータを送信してそのデータをコンソールに出してみよう

まずは最も単純にkafkaにデータをコマンドラインで送信しそのデータを読み取ってconsole出力してみましょう

cmd -> kafka <- spark streaming(df) -> console

In [ ]:
# PySparkをKafkaと接続します(kafka <- spark streaming(df))
# ストリームの経路を作成行います。

# kafkaからデータを読み取る設定を行います。

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()

In [3]:
# ストリームから読み取って出力する設定(spark streaming(df) -> console)

console_stream = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("console") \
  .option("checkpointLocation", "/tmp/kafka/console/") \
  .start()

# ストリームから読み取ったデータのうちkeyとvalueを抜き出してStringにキャストしたのちconsoleに出力してね
# triggerは何秒単位で処理を行うか（処理というのは↑のconsoleに出力する処理のこと）今回は動画の都合で短め(おおよそ1~10分が多い)
# checkpointlocationは、処理をすると「ここまで処理をしたよ！」という情報が出力される場所のこと
# 仮にkafkaが落ちてもチェックポイントのデータが生きていれば復旧した後に途中からやり直しをすることが可能になる。

#root
# |-- key: binary (nullable = true)   
# |-- value: binary (nullable = true)
# |-- topic: string (nullable = true)
# |-- partition: integer (nullable = true)
# |-- offset: long (nullable = true)
# |-- timestamp: timestamp (nullable = true)
# |-- timestampType: integer (nullable = true)

# binary型なのは様々な型のデータが入ってくることが想定されるから

NameError: name 'df' is not defined

# 送信コマンド(cmd -> kafka)

```

/home/pyspark/kafka/bin/kafka-console-producer.sh \
   --topic pyspark-topic1  \
   --bootstrap-server kafka:9092 \
   --property parse.key=true --property key.separator=,

```

key.separatorはキーバリューを分割するためのオプションです、今回はカンマで区切るとキーとバリューを送ることが可能です。


In [ ]:
# ストリームの停止
console_stream.stop()

# ファイルシンクしてみよう
次は、コンソールではなくファイル内にシンクをしてみましょう

cmd -> kafka <- spark streaming(df) -> parquet(ファイル)

In [ ]:
# ストリームから読み取って出力する設定(spark streaming(df) -> file)

file_stream = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("parquet") \
  .option("path", "/tmp/data/") \
  .outputMode("append") \
  .partitionBy("key") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/parquet/") \
  .start()

# ほぼフルフルでのオプションです(必要に応じてオプションを付け足したりします)
# formatはparqeut
# pathはどこに出力するか
# 追記か？overwrite(上書きか)
# partitionbyパーティションを付ける場合
# 読み込みの状況を途中保存するためのチェックポイント

# 単純なテキストがparquetに変換されて出力されます。

In [ ]:
read_df=spark.read.parquet("/tmp/data/*")